In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy
import os.path
import pandas
from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from simple_dispatch import generatorDataShort

# eGRID Data

In [2]:
egrid_data_xlsx = '../Data/GridInputData/eGRID2019_data.xlsx'

egrid_unt = pandas.read_excel(egrid_data_xlsx, 'UNT'+'19', skiprows=[0])
egrid_unt['orispl_unit'] = egrid_unt.ORISPL.map(str) + '_' + egrid_unt.UNITID.map(str)

egrid_gen = pandas.read_excel(egrid_data_xlsx, 'GEN'+'19', skiprows=[0])
egrid_gen['orispl_unit'] = egrid_gen['ORISPL'].map(str) + '_' + egrid_gen['GENID'].map(str)

egrid_plnt = pandas.read_excel(egrid_data_xlsx, 'PLNT'+'19', skiprows=[0])


In [3]:
egrid_data_xlsx_new = '../Data/GridInputData/eGRID2022_data.xlsx'

egrid_unt_new = pandas.read_excel(egrid_data_xlsx_new, 'UNT'+'22', skiprows=[0])
egrid_unt_new['orispl_unit'] = egrid_unt_new.ORISPL.map(str) + '_' + egrid_unt_new.UNITID.map(str)

egrid_gen_new = pandas.read_excel(egrid_data_xlsx_new, 'GEN'+'22', skiprows=[0])
egrid_gen_new['orispl_unit'] = egrid_gen_new['ORISPL'].map(str) + '_' + egrid_gen_new['GENID'].map(str)

egrid_plnt_new = pandas.read_excel(egrid_data_xlsx_new, 'PLNT'+'22', skiprows=[0])


# Start from eGRID lists of retirements

In [4]:
egrid_gen = egrid_gen.loc[(~(egrid_gen['FUELG1'].isin(['SUN', 'MWH', 'WAT', 'WND'])))&(egrid_gen['ORISPL'].isin(egrid_plnt[egrid_plnt['NERC']=='WECC']['ORISPL'].values))].reset_index(drop=True)
egrid_gen_new = egrid_gen_new.loc[(~(egrid_gen_new['FUELG1'].isin(['SUN', 'MWH', 'WAT', 'WND'])))&(egrid_gen_new['ORISPL'].isin(egrid_plnt_new[egrid_plnt_new['NERC']=='WECC']['ORISPL'].values))].reset_index(drop=True)


In [5]:
unit_drops_2019egrid = egrid_gen.loc[(~egrid_gen['GENYRRET'].isna())].loc[:, ['PNAME', 'ORISPL', 'orispl_unit', 'GENYRRET']].reset_index(drop=True)
unit_drops_2022egrid = egrid_gen_new.loc[(~egrid_gen_new['GENYRRET'].isna())].loc[:, ['PNAME', 'ORISPL', 'orispl_unit', 'GENYRRET']].reset_index(drop=True)

In [6]:
unit_drops_merged = unit_drops_2019egrid.merge(unit_drops_2022egrid, on='orispl_unit', how='outer')
unit_drops_merged

,PNAME_x,ORISPL_x,orispl_unit,GENYRRET_x,PNAME_y,ORISPL_y,GENYRRET_y
0,Cholla,113.0,113_4,2020.0,NaN,NaN,NaN
1,Cholla,113.0,113_2,2015.0,NaN,NaN,NaN
2,Coronado,6177.0,6177_CO1,2032.0,Coronado,6177.0,2032.0
3,Coronado,6177.0,6177_CO2,2032.0,Coronado,6177.0,2032.0
4,Crosscut,143.0,143_CC1,1974.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
719,NaN,NaN,4162_3,NaN,Naughton,4162.0,2029.0
720,NaN,NaN,54374_2004,NaN,Sinclair Oil Refinery,54374.0,2022.0
721,NaN,NaN,54374_2006,NaN,Sinclair Oil Refinery,54374.0,2022.0
722,NaN,NaN,54374_2008,NaN,Sinclair Oil Refinery,54374.0,2022.0


In [7]:
unit_drops_cleaned = unit_drops_merged.loc[:, ['orispl_unit']].copy()

for i in unit_drops_merged.index:
    if unit_drops_merged.loc[i, ['PNAME_x', 'ORISPL_x', 'GENYRRET_x']].isna().sum() == 0:
        # in 2019 data set
        if unit_drops_merged.loc[i, ['PNAME_y', 'ORISPL_y', 'GENYRRET_y']].isna().sum() == 0:
            # also in 2022 data set
            unit_drops_cleaned.loc[i, 'WECC Name'] = unit_drops_merged.loc[i, 'PNAME_x']
            unit_drops_cleaned.loc[i, 'orispl'] = unit_drops_merged.loc[i, 'ORISPL_x']
            unit_drops_cleaned.loc[i, 'retirement_year'] = unit_drops_merged.loc[i, 'GENYRRET_y'] # more up to date
            unit_drops_cleaned.loc[i, 'egrid_source'] = 2022
        else:
            if unit_drops_merged.loc[i, 'GENYRRET_x'] > 2022:
                print('Check ', i, unit_drops_merged.loc[i, 'orispl_unit'])
            # only in 2019 data set
            unit_drops_cleaned.loc[i, 'WECC Name'] = unit_drops_merged.loc[i, 'PNAME_x']
            unit_drops_cleaned.loc[i, 'orispl'] = unit_drops_merged.loc[i, 'ORISPL_x']
            unit_drops_cleaned.loc[i, 'retirement_year'] = unit_drops_merged.loc[i, 'GENYRRET_x'] 
            unit_drops_cleaned.loc[i, 'egrid_source'] = 2019
    elif unit_drops_merged.loc[i, ['PNAME_y', 'ORISPL_y', 'GENYRRET_y']].isna().sum() == 0:
        # only in 2022 data set
        unit_drops_cleaned.loc[i, 'WECC Name'] = unit_drops_merged.loc[i, 'PNAME_y']
        unit_drops_cleaned.loc[i, 'orispl'] = unit_drops_merged.loc[i, 'ORISPL_y']
        unit_drops_cleaned.loc[i, 'retirement_year'] = unit_drops_merged.loc[i, 'GENYRRET_y']
        unit_drops_cleaned.loc[i, 'egrid_source'] = 2022
    else:
        print('Check ', i, unit_drops_merged.loc[i, 'orispl_unit'])
        
unit_drops_cleaned = unit_drops_cleaned.rename(columns={'orispl_unit':'orispl_unit_gen'})

Check  540 7790_1


In [8]:
unit_drops_cleaned.loc[540]

orispl_unit_gen     7790_1
WECC Name          Bonanza
orispl              7790.0
retirement_year     2030.0
egrid_source        2019.0
Name: 540, dtype: object

In [9]:
# unit_drops_merged.loc[540] # was this retired early or was the retirement cancelled?
# # checked: previous retirement year 2030 is still in the wecc assessment 2022, so keep it

### Fix Names as units named differently in gen vs unt tabs
gen to unt mapping, based on 2021 egrid

In [10]:

orispl_to_fix = {141:{'AF1':'1', 'AF2':'2', 'AF3':'3'},
                 160:{'ST1':'1', 'ST2':'2', 'ST3':'3', 'GT4':'4'},
                 56948:{str(k):'CT'+str(k) for k in np.arange(1, 13)},
                 6177:{'CO1':'U1B', 'CO2':'U2B'},
                 124:{'GT2':'GT1'},
                 55129:{'CTG1':'DBG1', 'CTG2':'DBG2'},
                 60768:{'ST10':'CTG3'},
                 55306:{'CTG1':'1CTGA', 'CTG2':'1CTGB', 'CTG3':'2CTGA', 'CTG4':'2CTGB',
                        'CTG5':'3CTGA', 'CTG6':'3CTGB', 'CTG7':'4CTGA', 'CTG8':'4CTGB'},
                 55124:{'CTG1':'P1', 'CTG2':'P2'},
                 126:{'ST3':'3'},
                 147:{'KY7':'K-7'},
                55481:{'GT1':'1', 'GT2':'2', 'GT3':'5', 'GT6':'6'},
                56616:{'G3':'4PB'},
                 55455:{'CT1A':'CC1A', 'CT1B':'CC1B', 'CT2A':'CC2A', 'CT2B':'CC2B'},
                 118:{'GE1':'CT3'},
                 8068:{'ST'+str(k):str(k) for k in ['5A', '5B', '6A']},
                 55522:{'CT'+str(k):'CT0'+str(k) for k in np.arange(1, 10)},
                 117:{'C4-1':'CC4', 'C5-1':'CC5A', 'C5-2':'CC5B'},
                 120:{'ST1':'1', 'GT5':'CT5', 'GT6':'CT6'},
                 54694:{'GEN1':'4101'},
                 62115:{'1S':'HRSG1A'},
                 62116:{'1S':'HRSG1A'},
                 55951:{'AMP1':'AMP-1'},
                 57564:{'CTG':'8'},
                 56706:{'AMPC':'SD200'},
                 10684:{'TG5':'BLR22', 'TG8':'BLR25', 'TG9':'BLR26'},
                 10650:{'GEN1':'GT1'},
                 56474:{'1':'CT1'},
                 10649:{'GEN1':'GT1'},
                 10764:{'GEN1':'Zurn'},
                 55295:{'CT1':'1', 'CT2':'2'},
                 10652:{'GEN1':'BLR1'},
                 302:{'CEC '+str(k):str(k) for k in np.arange(6, 11)},
                 55508:{'CPP2':'GT-1', '8':'GT-2'},
                 55510:{'CPP4':'GT-1'},
                 55513:{'CPP7':'GT-1'},
                 55499:{'CPP1':'GT-1'},
                 57027:{'CPP'+str(k):str(k) for k in np.arange(1, 5)},
                 10169:{'GEN1':'D1'},
                 56475:{'1':'CT1'},
                 55540:{'GEN1':'1A'},
                 10623:{'GEN1':'1', 'GEN2':'2'},
                 50131:{'K100':'1'},
                 10661:{'GEN5':'4'},
                 56532:{'A':'CT1', 'B':'CT2'},
                 10840:{'GEN1':'U1', 'GEN2':'U2'},
                 50632:{'GEN1':'1'},
                 55625:{'CT1':'UNIT1'},
                 55512:{'CPP6':'GT-1'},
                 55333:{'CTG'+str(k):str(k) for k in [1,2,3]},
                 10300:{'GEN1':'BLR1'},
                 60100:{'G-1':'B-1'},
                 56026:{'CTG1':'PCT1', 'CTG1':'PCT2'},
                 260:{'CT1A':'1A', 'CT1B':'2A', 'CT2A':'3A', 'CT2B':'4A'},
                 389:{'2A':'2-2', '31':'3-1', '32':'3-2'},
                 56707:{'AMPC':'SD200'},
                 55400:{'CTG1':'CTG-1', 'CTG2':'CTG-2'},
                 50530:{'GEN3':'H43'},
                 10052:{'GEN1':'BLR1'},
                 55847:{'CTG1':'UNIT1'},
                 56476:{'A':'GT1', 'B':'GT2'},
                 55810:{'S3':'S-3', 'S4':'S-4', 'S5':'S-5'},
                 11034:{'GEN1':'S-100'},
                 54749:{'CTG':'1'},
                 55627:{'CT1':'UNIT1'},
                 10349:{'GEN1':'2'},
                 399:{'10A':'**10A', '10B':'**10B', 'GT10':'10', 'GT11':'11', 'GT12':'12', 'GT13':'13', 'GT14':'14'},
                 50541:{'ST1':'HR200', 'ST2':'HRSG1'},
                 10777:{'GEN1':'ZURN1'},
                 50049:{'GEN1':'BLRA', 'GEN2':'BLRB', 'GEN3':'BLRC'},
                 55541:{'CTG'+str(k):str(k) for k in [1,2,3]},
                 10294:{'STG':'2'},
                 10405:{'GEN1':'1'},
                 55151:{'GEN'+str(k):'CTG-'+str(k) for k in [1,2,3,4]},
                 7987:{'L1':'01'},
                 55626:{'CT1':'UNIT1'},
                 55542:{'CTG1':'1', 'CTG2':'2'},
                 54768:{'GEN1':'GT1'},
                 341:{'CT'+str(k):str(k) for k in [1,2,3,4]},
                 55217:{'CTG1':'X724', 'CTG2':'X725'},
                 10342:{'TG3':'EIHRSG'},
                 57483:{'CTG'+str(k):'GT-'+str(k) for k in [1,2,3,4]},
                 54912:{'STG1':'HRSG1'},
                 56471:{'1':'CT1'},
                 50612:{'GEN1':'GT1'},
                 55393:{'CTG1':'1', 'CTG2':'2'},
                 56473:{'1':'CT1'},
                 358:{'MV3A':'3-1', 'MV3B':'3-2', 'MV4A':'4-1', 'MV4B':'4-2'},
                 54626:{'TG01':'BL01'},
                 6013:{'O1':'01', 'O2':'02'},
                 55345:{'1-01':'CTG-1', '1-02':'CTG-2'},
                 50560:{'GEN1':'B101'},
                 55656:{'CT01':'CT001', 'CT02':'CT002', 'CT04':'CT004'},
                 10472:{'GEN1':'B300'},
                 10767:{'GEN1':'CFB'},
                 10772:{'GEN1':'CFB'},
                 55963:{'CTG1':'1'},
                 56469:{'1':'FW-1'},
                 56298:{'0001':'CT001','0002':'CT002'},
                 56467:{'CTG1':'CT1', 'CTG2':'CT2'},
                 50865:{'K100':'1'},
                 50864:{'K100':'1'},
                 7551:{'CT1A':'1A', 'CT1B':'1B', 'CT1C':'1C'},
                 57482:{'CTG'+str(k):str(k) for k in np.arange(1, 9)},
                 50110:{'GEN1':'BLR1'},
                 10144:{'GEN4':'BLR3'},
                 50112:{'GEN3':'BLR1'},
                 50837:{'GEN1':'UNIT1'},
                 7552:{'CCCT':'1'},
                 59658:{'GEN1':'BLR1'},
                 60698:{'GT1':'D1', 'GT2':'D7'},
                 54238:{'STG':'BIOMS1'},
                 55182:{'X718':'CTG1', 'X719':'CTG2'},
                 50881:{'GEN'+str(k):'BLR'+str(k) for k in [1,2,3]},
                 55855:{'CTG1':'UNIT1'},
                 7266:{'NA1':'1'},
                 10836:{'GEN1':'BLR1'},
                 55200:{'UN5':'CT5', 'UN6':'CT6'},
                 55645:{'CT01':'CT-01', 'CT02':'CT-02'},
                 10003:{'GEN1':'BLR1', 'GEN2':'BLR2'},
                 6021:{'1':'C1', '2':'C2', '3':'C3'},
                 55453:{'S'+str(k):str(k) for k in [1,2,3,4,5,6]},
                 525:{'1':'H1', '2':'H2'},
                 50707:{'LMA':'S001', 'LMB':'S002', 'LMC':'S003', 'LMD':'S004', 'LME':'S005'},
                 55127:{'UN1':'CT1', 'UN2':'CT2'},
                 56998:{'GT1':'CT01', 'GT2':'CT02', '4':'CT04', '5':'CT05', '6':'CT06', '7':'CT07'},
                 6761:{'1':'101'},
                 8219:{'GT1':'2', 'GT2':'3'},
                 55835:{'CTG1':'1', 'CTG2':'2'},
                 56445:{'GEN1':'CT-01', 'GEN2':'CT-02'},
                 55207:{'UN7':'CT7', 'UN8':'CT8'},
                 10504:{'1500':'SB1', '2500':'SB2'},
                 55744:{'1':'CT01'},
                 50637:{'GEN1':'1PWR', 'GEN2':'2PWR', 'GEN3':'4PWR', 'GEN4':'5REC'},
                 7953:{'1':'CT1', '2':'CT2', '3':'CT3'},
                 57028:{'GTG':'CT1'},
                 55179:{'CTG1':'CTGEN1'},
                 10784:{'GEN1':'CBLR'},
                 56908:{'1':'1A', '2':'2A', '3':'3A'},
                 55749:{'UNT1':'U1'},
                 58284:{'0001':'1'},
                 55514:{'CTG1':'CTG01', 'CTG2':'CTG02'},
                 55322:{'CTG1':'CTG-1', 'CTG2':'CTG-2', 'CTG3':'CTG-3', 'CTG4':'CTG-4'},
                 2322:{str(k):str(k)+'A' for k in np.arange(11, 23)},
                 55077:{'ED01':'EDE1', 'ED02':'EDE2', 'EDO1':'EDE1', 'EDO2':'EDE2'},
                 7082:{'GT1':'**3', 'GT4':'**4', '5':'**5', '6':'**6'},
                 55687:{'A01':'BHG1', 'A02':'BHG2'},
                 10761:{'GEN'+str(k):str(k) for k in [1,2,3,4,5]},
                 54350:{'STM':'AUX'},
                 54349:{'STM':'AUX'},
                 54271:{'STG':'HRSG1'},
                 55841:{'CT1':'A01', 'CT2':'A03'},
                 54854:{'5657':'3', '5658':'4', '5659':'5'},
                 10869:{'GE':'NORTH', 'GE-2':'SOUTH'},
                 58503:{'GEN1':'CTEU1'},
                 50650:{'GEN1':'BLR1'},
                 7350:{'1':'CTG1', '2':'CTG2'},
                 7931:{'2':'SG02'},
                 50396:{'GEN'+str(k):'BLR'+str(k) for k in [1, 2, 6]},
                 58109:{'TG1':'REC1', 'TG2':'REC2'},
                 56192:{'1':'FBB'},
                 54761:{'GEN1':'1', 'GEN2':'2'},
                 55328:{'CTG1':'CTG-1', 'CTG2':'CTG-2'},
                 50191:{'TG4':'REC4'},
                 56227:{'1':'PWEU1'},
                 55478:{'0001':'CT1'},
                 3456:{'5CT1':'**4', '5CT2':'**5', 'CT1':'GT-6A', 'CT2':'GT-6B'},
                 7790:{'1':'1-1'},
                 56102:{'CT1A':'CTG1A', 'CT1B':'CTG1B'},
                 6481:{'1':'1SGA', '2':'2SGA'},
                 56253:{'MC1':'MC-1', 'MC2':'MC-2'},
                 56177:{'GT1':'U1'},
                 50951:{'GEN1':'1'},
                 50185:{'TG1':'1RB', 'TG2':'2RB'},
                 7870:{'CTG1':'CT1', 'CTG2':'CT2', 'CTG3':'CT3'},
                 54537:{'CT1A':'CT-1A', 'CT1B':'CT-1B'},
                 607:{'3':'CT3', '4':'CT4'},
                 55818:{'FICT':'F1CT'},
                 55482:{'G1':'CT-1'},
                 7999:{'CT1':'1', 'CT2':'2'},
                 55882:{'GEN1':'BLR1'},
                 50886:{'GEN1':'1-1A'},
                 54476:{'GEN1':'CT-1'},
                 3845:{'2':'BW22'},
                 57703:{'01A':'CT01', '01B':'CT02', '02A':'CT03'},
                 4158:{str(k):'BW4'+str(k) for k in [1,2,3,4]},
                 54318:{'TG1':'C', 'TG2':'D'},
                 57915:{str(k):str(k)+'BLR' for k in [1,2,3,4,5,6]},
                 8066:{str(k):'BW7'+str(k) for k in [1,2,3,4]},
                 55477:{'GT2':'CT2'},
                 7504:{'2':'001'},
                 55479:{'0001':'001'},
                 56319:{'0001':'001'},
                 56596:{'5':'001'},
                 6101:{'1':'BW91'},
                 9:{'1':'CTG-1'},
                 57099:{'STG1':'4RB'},
                 126:{'1':'ST1', '2':'ST2'}, 
                 4941:{'NAV1':'1', 'NAV2':'2', 'NAV3':'3'},
                 10213:{'GEN3':'HRSG1', 'GEN4':'HRSG2'},
                 58083:{'STG1':'HRSG'},
                 10350:{'GEN1':'1'},
                 50112:{'GEN1':'BLR1', 'GEN2':'BLR2'},
                 6106:{'1':'1SG'},
                 58352:{'G-11':'B-11'},
                 3845:{'1':'BW21', '2':'BW22'}}

In [11]:
unit_drops_cleaned['orispl_unit_unt'] = unit_drops_cleaned['orispl_unit_gen'].copy()
for key1, val1 in orispl_to_fix.items():
    if key1 in unit_drops_cleaned['orispl'].values:
        for key2, val2 in val1.items():
            orispl_unt_val_gen = str(key1)+'_'+key2
            orispl_unt_val_unt = str(key1)+'_'+val2
            if orispl_unt_val_gen in unit_drops_cleaned['orispl_unit_gen'].values:
                ind = unit_drops_cleaned.loc[unit_drops_cleaned['orispl_unit_gen']==orispl_unt_val_gen].index
                unit_drops_cleaned.loc[ind, 'orispl_unit_unt'] = orispl_unt_val_unt

# Check for things in 2022 Resource Assessment that aren't on this list

Must be done manually because names don't match between WECC document and eGRID.

File downloaded from https://www.wecc.org/Reliability/2022%20Western%20Assessment%20of%20Resource%20Adequacy.pdf

In [15]:
fillcols = ['WECC Name', 'orispl', 'orispl_unit_gen', 'retirement_year']
added_2022doc = pd.DataFrame(columns=fillcols)

In [16]:
added_2022doc.loc[len(added_2022doc), fillcols] = ['TMF Biofuels Digester', '58382', '58382_GEN1', 2023]
added_2022doc.loc[len(added_2022doc), fillcols] = ['TMF Biofuels Digester', '58382', '58382_GEN2', 2023]
added_2022doc.loc[len(added_2022doc), fillcols] = ['TMF Biofuels Digester', '58382', '58382_GEN3', 2023]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Sunnyside Cogeneration Associates', '50951', '50951_1', 2023]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Arapahoe 5', '55200', '55200_CT5', 2023]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Arapahoe 6', '55200', '55200_CT6', 2023]
for ind in added_2022doc[added_2022doc['WECC Name'].str.contains('Diablo')].index: # new diablo canyon retirement date
    added_2022doc.loc[ind, 'retirement_year'] = 2030
added_2022doc.loc[len(added_2022doc), fillcols] = ['Tracy 3', '2336', '2336_3', 2024]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Tracy G240', '2336', '2336_4', 2024]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Hill Air Force Base (AFB)', '60349', '60349_LFGG1', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Hill Air Force Base (AFB)', '60349', '60349_LFGG2', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Douglas County Forest Products (Douglas County Inc)', '61761', '61761_GEN-1', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Cholla', '113', '113_1', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Cholla', '113', '113_3', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Colo. Power Partners 1', '10682', '10682_GT2', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Colo. Power Partners 2', '10682', '10682_GT4', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Colo. Power Partners 3', '10682', '10682_GT5', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Beowawe', '10287', '10287_GEN1', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Harry Allen 1', '7082', '7082_**3', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Colstrip 3', '6076', '6076_3', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Colstrip 4', '6076', '6076_4', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Colstrip Energy', '10784', '10784_BLR1', 2025]

added_2022doc.loc[len(added_2022doc), fillcols] = ['Sun Peak 3', '54854', '54854_5657', 2026]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Sun Peak 4', '54854', '54854_5658', 2026]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Sun Peak 5', '54854', '54854_5659', 2026]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Biomass One, L.P. 2', '10869', '10869_NORTH', 2026]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Biomass One, L.P. 3', '10869', '10869_SOUTH', 2026]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Spindle Hill 1', '56445', '56445_CT-01', 2027]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Spindle Hill 2', '56445', '56445_CT-02', 2027]
added_2022doc.loc[len(added_2022doc), fillcols] = ['North Loop 1', '86088', '6088_1', 2027]
added_2022doc.loc[len(added_2022doc), fillcols] = ['North Loop 2', '86088', '6088_2', 2027]
added_2022doc.loc[len(added_2022doc), fillcols] = ['North Loop 3', '86088', '6088_3', 2027]

print('All 34 plains end:', len(egrid_gen.loc[(egrid_gen['PNAME'].str.contains('Plains End'))]))
tmp = egrid_gen.loc[(egrid_gen['PNAME'].str.contains('Plains End'))]
for i in tmp.index:
    added_2022doc.loc[len(added_2022doc), fillcols] = ['Plains End '+tmp.loc[i, 'GENID'], str(tmp.loc[i, 'ORISPL']), tmp.loc[i, 'orispl_unit'], 2028]
    
added_2022doc.loc[len(added_2022doc), fillcols] = ['Roseburg Forest Products', '56469', '56469_1', 2028]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Las Vegas Generating Station GEN1', '10761', '10761_GEN1', 2029]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Las Vegas Generating Station GEN2', '10761', '10761_GEN2', 2029]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Hill Air Force Base (AFB)', '60349', '60349_LFGG1', 2025]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Hill Air Force Base (AFB)', '60349', '60349_LFGG2', 2025]


 # COP has a different name in wecc docs and egrid
# egrid_gen.loc[(egrid_gen['GENYRONL']==1980)&(np.abs(egrid_gen['NAMEPCAP'] - 87) < 5)]
# added_2022doc.loc[len(added_2022doc), fillcols] = ['COP', '9', '9_CTG-1', 2030]

added_2022doc.loc[len(added_2022doc), fillcols] = ['Bonanza', '7790', '7790_1', 2030]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Comanche 3', '470', '470_3', 2030]

added_2022doc.loc[len(added_2022doc), fillcols] = ['Clark 4', '2322', '2322_GT4', 2030]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Fountain Valley 1', '55453', '55453_1', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Fountain Valley 2', '55453', '55453_2', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Fountain Valley 3', '55453', '55453_3', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Fountain Valley 4', '55453', '55453_4', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Fountain Valley 5', '55453', '55453_5', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Fountain Valley 6', '55453', '55453_6', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Roseburg Landfill Gas Energy, LLC', '57911', '57911_1', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['H.W. Sundt 3', '126', '126_3', 2032]


# "H. Wilson Sundt Generating Station is a retired power station in Tucson, Pima, Arizona, United States. It is also known as Irvington Generating Station."
added_2022doc.loc[len(added_2022doc), fillcols] = ['Irvington or H. Wilson Sundt 1', '126', '126_GT1', 2032]
added_2022doc.loc[len(added_2022doc), fillcols] = ['Irvington or H. Wilson Sundt 2', '126', '126_GT2', 2032]

All 34 plains end: 34


In [17]:
added_2022doc['orispl_unit_unt'] = added_2022doc['orispl_unit_gen'].copy()

In [18]:
added_2022doc['orispl'] = added_2022doc['orispl'].astype(int)

In [19]:
added_2022doc_map = {54854:{'5657':'3', '5658':'4', '5659':'5'},
 56469:{'1':'FW-1'},
 10761:{'GEN1':'1', 'GEN3':'2', 'GEN4':'3', 'GEN5':'4', 'GEN6':'5'}, # gen2 excluded as different type, match by CT,
 7790:{'1':'1-1'}
 }

In [21]:
# unit_drops_cleaned['orispl_unit_unt'] = unit_drops_cleaned['orispl_unit_gen'].copy()
for key1, val1 in added_2022doc_map.items():
    if key1 in added_2022doc['orispl'].values:
#         print(key1)
        for key2, val2 in val1.items():
            orispl_unt_val_gen = str(key1)+'_'+key2
            orispl_unt_val_unt = str(key1)+'_'+val2
            if orispl_unt_val_gen in added_2022doc['orispl_unit_gen'].values:
                ind = added_2022doc.loc[added_2022doc['orispl_unit_gen']==orispl_unt_val_gen].index
                added_2022doc.loc[ind, 'orispl_unit_unt'] = orispl_unt_val_unt

In [23]:
added_2022doc

,WECC Name,orispl,orispl_unit_gen,retirement_year,orispl_unit_unt
0,TMF Biofuels Digester,58382,58382_GEN1,2023,58382_GEN1
1,TMF Biofuels Digester,58382,58382_GEN2,2023,58382_GEN2
2,TMF Biofuels Digester,58382,58382_GEN3,2023,58382_GEN3
3,Sunnyside Cogeneration Associates,50951,50951_1,2023,50951_1
4,Arapahoe 5,55200,55200_CT5,2023,55200_CT5
...,...,...,...,...,...
78,Fountain Valley 6,55453,55453_6,2032,55453_6
79,"Roseburg Landfill Gas Energy, LLC",57911,57911_1,2032,57911_1
80,H.W. Sundt 3,126,126_3,2032,126_3
81,Irvington or H. Wilson Sundt 1,126,126_GT1,2032,126_GT1


In [24]:
unit_drops = pd.concat((unit_drops_cleaned, added_2022doc), axis=0).loc[:, ['WECC Name', 'orispl', 'orispl_unit_unt', 'retirement_year']]
unit_drops = unit_drops.rename(columns={'orispl_unit_unt':'orispl_unit'})

In [26]:
for ind in unit_drops[unit_drops['orispl_unit'].isin(['3456_3', '3456_4'])].index:
    unit_drops.loc[ind, 'retirement_year'] = 2026 # retirement moved up for newman 3 and 4
unit_drops.loc[unit_drops.loc[unit_drops['orispl_unit']=='8219_1'].index, 'retirement_year'] = 2030 # update retirement year for ray d nixon 1

In [27]:
unit_drops.loc[unit_drops.loc[unit_drops['orispl_unit']=='525_H2'].index, 'retirement_year'] = 2028 # update retirement year for hayden 2
unit_drops.loc[unit_drops.loc[unit_drops['orispl_unit']=='8066_BW72'].index, 'retirement_year'] = 2028 # update retirement year for jim bridger 2

In [28]:
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Oakland', '6211', '6211_GEN1', 2023] # added and updated based on news
# unit_drops.loc[len(unit_drops), fillcols] = ['Oakland', '6211', '6211_GEN2', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Oakland', '6211', '6211_GEN3', 2023]


/var/folders/x0/47mhggk13ms4ct1vkkn6b88r0000gs/T/ipykernel_4609/4281721601.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '6211' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Oakland', '6211', '6211_GEN1', 2023] # added and updated based on news


In [29]:
inds_drop = unit_drops[unit_drops['WECC Name'].str.contains('Scattergood')].index # should remove - these plans paused
unit_drops = unit_drops.drop(inds_drop, axis=0)


Couldn't find: WM Landfill (3.2 MW) - no matches and too small to find more information on name; Tata Chemicals (30 MW coal added 2020 removed 2023) - likely wasn't online yet for 2021 egrid due to 2020 delays; Deseret Generation & Transmission (Bonanza) (DGT EEI) 100 MW added in 2020 removed in 2024 - likely wasn't online yet for 2021 egrid due to 2020 delays.
Checked in 2022 egrid and no news of these two plants either.

In [37]:
unit_drops.to_csv('../Data/scheduled_retirements_2024final.csv')